In [1]:
#!conda install pytorch==1.7.1 torchvision
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-gbh7i_yw
  Running command git clone --filter=blob:none -q https://github.com/openai/CLIP.git /tmp/pip-req-build-gbh7i_yw
  Resolved https://github.com/openai/CLIP.git to commit 40f5484c1c74edd83cb9cf687c6ab92b28d8b656
  Preparing metadata (setup.py) ... done


### 多模态商品分类

参考实现 https://shopify.engineering/using-rich-image-text-data-categorize-products 

需求是将商品输入图片结合商品的名称/描述,进行多模态的模型多标签输出.
可以支持
- 0样本的场景
- 小样本的场景
- 层级分类的场景

范例输入来自 https://www.hidizs.net/products/ap80-pro-x-fully-balanced-lossless-music-player-ayfk?spm=..collection.header_1.1&spm_prev=..product.header_1.1 

如下图片, 是一个商品名称为 `Hidizs AP80 PRO-X Portable Balanced Lossless MQA Music Player`,正确分类应该为`Portable Music Player`

![pic](../1.png)

In [43]:
import torch
import clip
from PIL import Image

#preload model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

#input define
input_classes = ["Bundles","Earphone","Portable Music Player","Bluetooth"]
#product_name = "Hidizs AP80 PRO-X Portable Balanced Lossless MQA Music Player"
product_name = "Hidizs AP80 PRO-X Balanced Lossless MQA"
image_path = "../1.png"

def infer_single(input_classes,product_name,image_path):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    #tokenize
    text_class = clip.tokenize(input_classes).to(device)
    text_input = clip.tokenize(product_name).to(device)
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    #encode
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_input)
    #here we use sum, we can use a agg function if we got training data
    agg_feature = image_features + text_features
    text_features = model.encode_text(text_class)
    #get highest class
    similarities = (agg_feature @ text_features.T).squeeze(1)
    best_photo_idx = (-similarities).argsort()
    result = input_classes[best_photo_idx[0][0]]
    print ("result class: ",result)
    return

In [44]:
infer_single(input_classes,product_name,image_path)

result class:  Portable Music Player
